In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from google.colab import drive
drive.mount('/content/drive')

# Aim is to predict the marks of students of the test data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
# Use the file namd 'training data' to train the model

data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Training data.xlsx')
x_train = np.array(data.iloc[:,0:8])
y_train = np.array(data.iloc[:,8]).reshape(-1,1)

# Try plotting y_train with different features
# To get an idea whether to add some features or not
# Add some features if required in x_train

# Also do label encoding for features not represented in numbers
# refer the link if not know : https://youtu.be/589nCGeWG1w?si=t2Wa7LgbUOO4RooM

def feature_changing(x_train):
  new_feature = x_train[:,3] / x_train[:,4]
  x_train = np.concatenate((x_train,new_feature.reshape(-1,1)),axis=1)
  return x_train

x_train = feature_changing(x_train)

In [47]:
def z_score(x_train):

  x_mean = np.mean(x_train,axis=0)
  x_std = np.std(x_train,axis=0)

  x_train = (x_train - x_mean)/x_std


  return x_train,x_std,x_mean

In [42]:
def cost(x_train,y_train,w,b):

    m = x_train.shape[0]
    loss = 0.0
    for i in range(m):
        f_wb = np.dot(x_train[i],w) + b
        loss += (f_wb - y_train[i])**2
    loss = loss/(2*m)

    return loss

In [43]:
def compute_gradient(x_train,y_train,w,b):

    m,n = x_train.shape
    grad_w = np.zeros((n,1))
    grad_b = 0.0
    for i in range(m):
        err = (np.dot(x_train[i],w) + b) - y_train[i]
        for j in range(n):
            grad_w[j] += err * x_train[i,j]
        grad_b += err

    grad_w = grad_w/m
    grad_b = grad_b/m

    return grad_w,grad_b




In [55]:
def gradient_descent(x_train,y_train,w,b):
  w = copy.deepcopy(w)
  b = b
  alpha = 0.05
  for i in range(100):
    grad_w,grad_b = compute_gradient(x_train,y_train,w,b)
    w = w - alpha*grad_w
    b = b - alpha*grad_b



  return w,b

In [66]:
x_train = x_train.astype(np.float64)
x_train,x_std,x_mean = z_score(x_train)

np.random.seed(2147483647)
w = np.random.randn(x_train.shape[1],1)
b = np.random.randn(1)

old_cost = 0


while abs(old_cost - cost(x_train,y_train,w,b))>0.00001:
  old_cost = cost(x_train,y_train,w,b)
  w,b = gradient_descent(x_train,y_train,w,b)


x_predict = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Test data.xlsx').iloc[:,:8].to_numpy()
x_predict = feature_changing(x_predict)
x_predict = (x_predict - x_mean)/x_std
ans = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Test data.xlsx').iloc[:,8].to_numpy()

y_predict = np.dot(x_predict,w) + b

accuracy = 0
for dim in range(len(ans)):
  if abs(y_predict[dim]-ans[dim])<0.5: # do not change the tolerance as you'll be checked on +- 0.5 error only
    accuracy += 1
accuracy = round(accuracy*100/200.0,2)
ok = 'Congratulations' if accuracy>95 else 'Optimization required'
print(f"{ok}, your accuracy is {accuracy}%")

Congratulations, your accuracy is 100.0%
